8. Feature Engineering für Regression

	•	Torquote: goals per 90 minutes.

	•	Assistquote: assists per 90 minutes.

	•	Kartenquote: Karten pro Spiel.

	•	Karriereintensität: Transfers / Spielzeit pro Jahr.